In [4]:
%load_ext autoreload 
%autoreload 2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as dists
import pytorch_lightning as pl
from torchvision.utils import make_grid
from experiments.vae import VAE
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from src.api import *
from src.utils import *


In [16]:

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((64, 64)),
])

data_train = datasets.MNIST(root="datasets/", train=True, transform=transform)
data_test = datasets.MNIST(root="datasets/", train=False, transform=transform)
train_loader = DataLoader(data_train, batch_size=48, shuffle=True, num_workers=8)

x, y = data_train[0]
x.size()




tensor(0.1038, grad_fn=<MseLossBackward>)

In [3]:
model = VAE(e, d, pz)
trainer = pl.Trainer(gpus=1, )
trainer.fit(model, train_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type  | Params
----------------------------------
0 | encoder | Chain | 879 K 
1 | decoder | Chain | 473 K 
----------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.414     Total estimated model params size (MB)


Training: -1it [00:00, ?it/s]

/home/xbt/anaconda3/envs/neuro/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1047: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
# z2_dim = 128
# z1_dim = 64

# mlp1 = build_mlp([x.numel(), 128, 128, z2_dim])
# mlp2 = build_mlp([z2_dim, 64, 64, 2*z1_dim])

# e = Chain([
#     mlp1,
#     Split(mlp2, nn.Linear(z2_dim, 2*z2_dim)),
#     Parallel(chunk(2, 1), chunk(2, 1)),
#     Parallel(make_normal, make_normal)
# ])

# q1, q2 = e(torch.flatten(x, 1))
# z1, z2 = q1.sample(), q2.sample()

# mlp3 = build_mlp([z1_dim, 64, z2_dim])
# mlp4 = build_mlp([2*z2_dim, 128, x.numel()])

# aux = []
# d = Chain([
#     Parallel(identity, mlp3),
#     lambda z2, z1: ((z2, z1), z1),
#     Parallel(Chain([Parallel(identity, nn.Linear(z2_dim, z2_dim)), cat(1), chunk(2, 1), make_normal, output(aux)]), identity),
#     Parallel(sample, identity),
#     cat(1),
#     mlp4,
#     reshape(-1, *x.size())
# ])

# x_ = d(z2, z1)
# F.mse_loss(x_, x[None])